Busque un ejemplo se utilice un algoritmo de K- Nearest, o árboles de decisión.



En este caso en empleo un Algoritmo K-Nearest para la clasificacion de documentos con el dataset de 20newgroups.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
import time

# inicio del proceso
print("Cargando el conjunto de datos...")
categories = ['sci.space', 'comp.graphics', 'rec.sport.hockey', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))
print(f'Datos cargados: {len(newsgroups.data)} documentos')

# Convertir los textos en vectores TF-IDF
print("Transformando los textos en vectores TF-IDF...")
vectorizer = TfidfVectorizer(stop_words='english', max_features=20000, ngram_range=(1,3))
X = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target
print(f'Tamaño de la matriz de características: {X.shape}')

# Reducir dimensionalidad con LSA
print("Reduciendo la dimensionalidad con LSA...")
svd = TruncatedSVD(n_components=500)
X_reduced = svd.fit_transform(X)
print(f'Tamaño de la matriz reducida: {X_reduced.shape}')

# Dividir en conjuntos de entrenamiento y prueba
print("Dividiendo datos en entrenamiento y prueba...")
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)
print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]} muestras')

# Modelo KNN
print("Entrenando modelo KNN...")
start_time = time.time()
knn = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='cosine')
knn.fit(X_train, y_train)
train_time = time.time() - start_time
print(f'Modelo entrenado en {train_time:.2f} segundos')

# Predicción y evaluación
print("Realizando predicciones...")
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión de KNN en clasificación de textos: {accuracy:.2f}')


Cargando el conjunto de datos...
Datos cargados: 3899 documentos
Transformando los textos en vectores TF-IDF...
Tamaño de la matriz de características: (3899, 20000)
Reduciendo la dimensionalidad con LSA...
Tamaño de la matriz reducida: (3899, 500)
Dividiendo datos en entrenamiento y prueba...
Tamaño del conjunto de entrenamiento: 3119 muestras
Tamaño del conjunto de prueba: 780 muestras
Entrenando modelo KNN...
Modelo entrenado en 0.00 segundos
Realizando predicciones...
Precisión de KNN en clasificación de textos: 0.88
